In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score
# import tensorflow as tf
from keras.models import load_model
import joblib
from config import interim_data_path, masks_path, data_path, sensors, targets

In [7]:
result = []
for sensor in sensors:
    X_tst = pd.read_feather(interim_data_path / f'{sensor}-tst.feather')
    y_tst = pd.read_feather(interim_data_path / 'tst_targets.feather') / 100

    scalers_path = data_path / 'scalers' / f'{sensor}_scaler.joblib'
    scaler = joblib.load(scalers_path)
    X_tst = pd.DataFrame(scaler.transform(X_tst), columns=X_tst.columns)

    masks_df = pd.read_feather(masks_path / f'{sensor}.feather')
    for target in targets:
        for to_mask in [False]:
            if to_mask == True:
                selected_columns = masks_df.index[masks_df[target]]
            elif to_mask == False:
                selected_columns = masks_df.index

            model_name = data_path / 'models' / 'NN' / f'NN_{sensor}_{target}.h5'
            model = load_model(model_name)
            y_pred = model.predict(X_tst[selected_columns])

            mae = mean_absolute_error(y_tst[target], y_pred)
            r2 = r2_score(y_tst[target], y_pred)
            result.append([sensor, target, to_mask, mae, r2])

20/20 [==============================] - 1s 73ms/step


In [9]:
pd.DataFrame(result)

,0,1,2,3,4
0,R1,NO_conc,False,0.039793,0.397417
1,R1,CH4_conc,False,0.053954,0.117288
2,R1,H2S_conc,False,0.013162,-27.749779
3,R1,SO2_conc,False,0.025098,-650.200408
4,R1,HCOH_conc,False,0.019418,-153.167462
...,...,...,...,...,...
103,R12,HCOH_conc,False,0.023194,-162.239276
104,R12,CO_conc,False,0.011680,-2.592932
105,R12,H2_conc,False,0.043910,0.183089
106,R12,NO2_conc,False,0.027446,-31.367783


In [5]:
X_tst

array([[0.3233666 , 0.32265959, 0.32072774, ..., 0.32264921, 0.32195855,
        0.32206591],
       [0.32303849, 0.32205109, 0.32046005, ..., 0.32254429, 0.32182782,
        0.32223583],
       [0.3227177 , 0.32195972, 0.32027542, ..., 0.3223872 , 0.32151472,
        0.32140729],
       ...,
       [0.05739289, 0.05728758, 0.05717312, ..., 0.05727651, 0.05721317,
        0.05721496],
       [0.05737305, 0.05721574, 0.05705563, ..., 0.05827133, 0.05824393,
        0.05831315],
       [0.05843293, 0.05829142, 0.05806709, ..., 0.05864726, 0.05858345,
        0.05863509]])

In [3]:
X_tst[selected_columns]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices